In [13]:
import gym
from envs.discrete_bs import BSEnv, decode_action, encode_action, encode_wealth, transform_Q_interval_to_Q_numeric, transform_Q_numeric_to_Q_interval, state_to_numeric
from envs import plotting

import numpy as np
import pandas as pd
import math
from collections import defaultdict
import itertools
import time
import matplotlib.pyplot as plt
from scipy import optimize

**Action space**  
Actions denote the fraction of wealth invested in the **risky asset**. Actions are discretized with a step size of 10%, i.e.

$$
\mathcal{A}=[0, 0.1, 0.2, \dots, 0.9, 1].
$$

In [14]:
# Define the vector of actions, discrete investment decisions in 10% steps
actions = np.arange(0, 1.01, step=0.1)                  
print("Actions (Investment in risky asset):", actions)  

Actions (Investment in risky asset): [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


**Wealth discretisation**

In [15]:
lower = 60      # upper limit of the lowest bucket (0, 90]
upper = 160     # lower limit of the highest bucket (upper, +inf)
delta_bin = 20  # bin-size inbetween 
wealth_bins = np.array([0] + np.arange(lower, upper+1, delta_bin).tolist() + [float('Inf')])  # +1 as upper limit is not included
#wealth_bins = np.arange(lower, upper+1, delta_bin)

print(pd.cut(x=[10], bins=wealth_bins, right=False, retbins=True)[0].categories)

IntervalIndex([[0.0, 60.0), [60.0, 80.0), [80.0, 100.0), [100.0, 120.0), [120.0, 140.0), [140.0, 160.0), [160.0, inf)],
              closed='left',
              dtype='interval[float64]')


**Discrete-time and discrete wealth Black-Scholes environment**

In [17]:
# Model parameters
'''
:params mu (float):         expected risky asset return
:params sigma (float):      risky asset standard deviation
:params r (float):          risk-less rate of return
:params T (float):          investment horizon
:params dt (float):         time-step size
:params V_0 (float, tuple): initial wealth, if tuple (v_d, v_u) draws initial wealth V(0) uniformly from [v_d, v_u]
:params actions (np.array): possible investment fractions into risky asset
:params wealth_bins (np.array): contains the limits of each wealth bin in ascending order
:params U_2 (callable):     utility function for terminal wealth (default log-utility)
'''

mu=0.06
sigma=0.2
r=0.04
T=2
dt=0.5
V_0=100
batch_size=1

env = BSEnv(mu=mu, sigma=sigma, r=r, T=T, dt=dt, V_0=V_0, actions=actions, wealth_bins=wealth_bins, batch_size=batch_size)

In [18]:
# Simulation of the wealth evolution in the BS market
done = False
print('Observation: {}, Exact_wealth: {}'.format(env.reset(), env.V_t))      # Reset the environment to state (0, 100)

while not done:
    print('Action: \n', decode_action(10, actions))
    next_obs, reward, done, info = env.step(10)              # Take the action 10 (i.e. 100% investment in risky asset) and observe the next state and reward
    print('Observation: {}, Reward: {}, Done: {}, Exact_wealth: {}'.format(next_obs, reward, done, env.V_t)),      

Observation: [[0.0 Interval(100.0, 120.0, closed='left')]], Exact_wealth: [100.]
Action: 
 1.0
Observation: [[0.5 Interval(100.0, 120.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 114.21643113261237
Action: 
 1.0
Observation: [[1.0 Interval(80.0, 100.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 96.14345955561608
Action: 
 1.0
Observation: [[1.5 Interval(80.0, 100.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 90.80025086692049
Action: 
 1.0
Observation: [[2.0 Interval(100.0, 120.0, closed='left')]], Reward: [4.62746225], Done: True, Exact_wealth: 102.25423893425943


**Epsilon-Greedy Policy**\
Source: https://www.geeksforgeeks.org/q-learning-in-python/#:~:text=Q%2DLearning%20is%20a%20basic,defined%20for%20states%20and%20actions.

In [19]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions): 
    """ 
    Creates an epsilon-greedy policy based 
    on a given Q-function and epsilon. 
       
    Returns a function that takes the state 
    as an input and returns the probabilities 
    for each action in the form of a numpy array  
    of length of the action space(set of possible actions). 
    """
    def policyFunction(state): 
   
        Action_probabilities = np.ones(num_actions, 
                dtype = float) * epsilon / num_actions 
                  
        best_action = np.argmax(Q[state]) 
        Action_probabilities[best_action] += (1.0 - epsilon) 
        return Action_probabilities 
   
    return policyFunction

In [20]:
# Action-value function parametrisation in a BS market with log-utility function an no consumption.
def action_func(x, a, b, c):
    return a * (x**2) + b * x + c


def update_action_direction(Q, state, action, actions, func, learning_rate, p0):
    '''
    Updates all action-values across action direction by fitting the parametrised function to the empirical 
    action-values and taking a step of size learning rate towards the fitted values.
    
    Args:
    :params Q [dict]: A dictionary containing the action-values
    :params state [Tuple]: The state to be updated.
    :params action [int]: The current action to be updated. It is not touched when calulating the concave update
    :params actions [np.array]: The possible investment choices.
    :params func [callable]: The parametrised functional form of the action-value function in direction of inv. in risky asset
    :params learning_rate [Float]: The step_size parameter.
    :params p0 [List(float)]: Initial parameters of func as starting point for the optimisation.
    
    Returns:
        - np.array with length equal to len(actions): Updated action-values for the current state for each action.
        - params [List(Float)]: List of function parameters.
    '''
    
    # Function values to fit to
    y = Q[state]
    # Optimize parameters a,b,c to fit to the function values f
    params, _ = optimize.curve_fit(func, actions, y, p0=p0)
    
    # predicted values for all actions
    y_fitted = func(actions, params[0], params[1], params[2])
    # difference between predicted values and actual values
    y_diff = y_fitted - y
    y_diff[action] = 0   # does not update the current action
    
    # Update step according to learning rate
    return y + learning_rate*y_diff, params

**Q-Learning Algorithm**  
The general Q-Learning Algorithm Outline can be found under the link below.  
Source: https://www.geeksforgeeks.org/q-learning-in-python/#:~:text=Q%2DLearning%20is%20a%20basic,defined%20for%20states%20and%20actions.

In [21]:
def qLearning(env, num_episodes, actions, conc_func, conc_learning_rate, concavisation_start, discount_factor = 1, alpha = 1, epsilon = 1): 
    """ 
    Q-Learning algorithm: Off-policy TD control. 
    Finds the optimal greedy policy while improving 
    following an epsilon-greedy policy
    
    Args:
    :params env [gym.environment]: The environement used in the algorithm to sample trajectories.
    :params num_episodes [Int]: Number of training episodes.
    :params actions [np.array]: Numpy array containing the allocation possibilities to the risky asset.
    :params conc_func [Callable]: Function parametrisation used for the update steps in action direction every iteration.
    :params conc_learning_rate [Float]: The step_size used for a step towards the fitted action-values in the update state in action direction.
    :params concavisation_start [Int]: Number of episode, when to start with the updates in action direction as well.
    :params discount_factor [Float]: The discount factor.
    :params alpha [Float]: The learning rate for Q-Learning (not used for now, as updates are 1/num_state_action_visits)
    :params epsilon [Float]: Random exploration factor in Q-Learning.
    """
       
    # Action value function 
    # A nested dictionary that maps 
    # state -> (action -> action-value). 
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    A = defaultdict(lambda: np.zeros(env.action_space.n))     # Dictionary: Counts visits of state-actions pairs
    P = defaultdict(lambda: [-1, 1, 1])                       # Dictionary: Contains parameters of conc_func for concavisation 
                                                              #             used as initial parameters for the next optimisation
   
    # Keeps track of useful statistics 
    stats = plotting.EpisodeStats( 
        episode_lengths = np.zeros(num_episodes), 
        episode_rewards = np.zeros(num_episodes))     
       
        
    # Create an epsilon greedy policy function 
    # appropriately for environment action space 
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n) 
       
        
    # For every episode
    returns=np.array([])
    terminal_wealths = np.array([])
    
    
    for ith_episode in range(num_episodes): 
           
            
        # Reset the environment and pick the first action 
        state = state_to_numeric(env.reset()[0])
        
        
        for t in itertools.count(): 
               
            # get probabilities of all actions from current state 
            action_probabilities = policy(state)
            # choose action according to  
            # the probability distribution 
            action = np.random.choice(np.arange( 
                      len(action_probabilities)), 
                       p = action_probabilities)
            A[state][action] += 1
   
            # take action and get reward, transit to next state 
            # next_states: np.array of length batch_size
            # rewards:     np.array of length batch_size
            # done: True/False
            next_states, rewards, done, _ = env.step(action)
            # next_states: np.array of length batch size, each entry is a tuple (t, mean.v)
            next_states = np.array([state_to_numeric(next_state) for next_state in next_states])
            # mean_reward: float
            mean_reward = np.mean(rewards)
            
   
            # Update statistics 
            stats.episode_rewards[ith_episode] += mean_reward 
            stats.episode_lengths[ith_episode] = t
               
            
            # TD Update 
            # Selects the batch_size next best actions for each next state in next_states
            best_next_actions = np.array([np.argmax(Q[(next_state[0], next_state[1])]) for next_state in next_states])
            # Calculates the Q-values for each of the following states
            next_q_values = np.array([Q[(next_state[0], next_state[1])][best_next_action] for next_state, best_next_action in zip(next_states, best_next_actions)])
            # Calculate the td-target as the mean reward + \beta * mean_next_q_values 
            # (the higher the batch_size the less noise in the target!!!)
            td_target = mean_reward + discount_factor * np.mean(next_q_values)
            # Calculate the delta
            td_delta = td_target - Q[state][action]


            # Update the action-value for the current state
            Q[state][action] += (1/A[state][action]) * td_delta          # Dynamic Learning Rate alpha=1/#visits of state-action pair
                                                                         # ensures convergence see Sutton & Barto eq. (2.7)

                
            
            # Monotone updates
            
            if td_delta > 0:
                states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] > state[1]) & (Q[s_tilde][action] < Q[state][action])]
                # get first state s_hat=(t, V_hat) with V_hat > V^* and Q(s_hat, action) >= Q[state, action]
                s_hat_list = sorted([s for s in Q.keys() if (s[0] == state[0]) & (s[1] > state[1]) & (Q[s][action] >= Q[state][action])], key=lambda x: x[1])
                
                
                # Update states in between via linear interpolation
                if len(s_hat_list) != 0:
                    s_hat = s_hat_list[0]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action] + (s_tilde[1] - state[1])*((Q[s_hat][action] - Q[state][action])/(s_hat[1] - state[1]))
               
                
                # No s_hat, hence update all s_tilde equally Q(s_tilde, action) = Q(s^*, action)
                else:
                    states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] > state[1])]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action]
            
            
            elif td_delta < 0:
                states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] < state[1]) & (Q[s_tilde][action] > Q[state][action])]
                # get first state s_hat=(t, V_hat) with V_hat < V^* and Q(s_hat, action) <= Q[state, action]
                s_hat_list = sorted([s for s in Q.keys() if (s[0] == state[0]) & (s[1] < state[1]) & (Q[s][action] <= Q[state][action])], key=lambda x: x[1], reverse=True)
                
                
                # Update states in between via linear interpolation
                if len(s_hat_list) != 0:
                    s_hat = s_hat_list[0]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[s_hat][action] + (s_tilde[1] - s_hat[1])*((Q[state][action] - Q[s_hat][action])/(state[1] - s_hat[1]))
                
                
                # No s_hat, hence update all s_tilde equally Q(s_tilde, action) = Q(s^*, action)
                else:
                    states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] < state[1])]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action]
                        
            else:
                pass
            
            
            # Updates in action direction (by parametrised function conc_func)
            # Starts after concavisation_start nr. of episodes
            #if ith_episode >= concavisation_start:
            #    # Updates all action-values for the current state (across risky asset allocation)
            #    new_action_values, params = update_action_direction(Q, state, action, actions, func=conc_func, learning_rate=conc_learning_rate, p0=P[state])
                # Update the Q-Table
            #    Q[state] = new_action_values
            #    # Save parameters of fitted function for the current state as starting point for the next optimisation
            #    P[state] = params
            
            
            
            # done is True if episode terminated    
            if done: 
                returns = np.append(returns, mean_reward)
                terminal_wealths=np.append(terminal_wealths, env.V_t)
                break
                   
            # Continue with the first next_state (simplifies, no need to average over wealths)        
            state = (next_states[0][0], next_states[0][1])
        
        if ith_episode % 10000 == 0:
            print("Episode: {}, Mean Return: {}, Mean Wealth (V_T): {}, Epsilon: {}".format(ith_episode, round(returns.mean(), 3), round(terminal_wealths.mean(), 3), epsilon))
            #print("td_delta:", td_delta)
            returns = np.array([])
            terminal_wealths=np.array([])
            
        # Epsilon-Decay    
        #if (ith_episode % 10000 == 0) & (ith_episode != 0):
        #    epsilon *= 0.98
        #    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n)
        #    alpha = 0.1
        
        # Alpha-Decay
        #if (ith_episode % 30000 == 0) & (ith_episode != 0):
        #    if alpha > 0.0011:
        #        alpha *= 1/10
            
       
    return Q, stats, A

In [ ]:
# Training of the Agent
num_episodes = 100000000   # Training for 100 mio. Episodes

# Starts timer
t_0 = time.time()

# Starts training
Q, stats, A = qLearning(env, num_episodes, actions, conc_func=action_func, conc_learning_rate = 1e-2, concavisation_start=10000)

# Ends timer
t_1 = time.time()

Episode: 0, Mean Return: 4.739, Mean Wealth (V_T): 114.331, Epsilon: 1
Episode: 10000, Mean Return: 4.693, Mean Wealth (V_T): 110.763, Epsilon: 1
Episode: 20000, Mean Return: 4.69, Mean Wealth (V_T): 110.353, Epsilon: 1
Episode: 30000, Mean Return: 4.692, Mean Wealth (V_T): 110.645, Epsilon: 1
Episode: 40000, Mean Return: 4.689, Mean Wealth (V_T): 110.286, Epsilon: 1
Episode: 50000, Mean Return: 4.69, Mean Wealth (V_T): 110.346, Epsilon: 1
Episode: 60000, Mean Return: 4.692, Mean Wealth (V_T): 110.65, Epsilon: 1
Episode: 70000, Mean Return: 4.693, Mean Wealth (V_T): 110.648, Epsilon: 1
Episode: 80000, Mean Return: 4.692, Mean Wealth (V_T): 110.616, Epsilon: 1
Episode: 90000, Mean Return: 4.69, Mean Wealth (V_T): 110.377, Epsilon: 1
Episode: 100000, Mean Return: 4.696, Mean Wealth (V_T): 111.054, Epsilon: 1
Episode: 110000, Mean Return: 4.692, Mean Wealth (V_T): 110.622, Epsilon: 1
Episode: 120000, Mean Return: 4.69, Mean Wealth (V_T): 110.377, Epsilon: 1
Episode: 130000, Mean Return: 4

Episode: 1090000, Mean Return: 4.692, Mean Wealth (V_T): 110.527, Epsilon: 1
Episode: 1100000, Mean Return: 4.691, Mean Wealth (V_T): 110.56, Epsilon: 1
Episode: 1110000, Mean Return: 4.69, Mean Wealth (V_T): 110.382, Epsilon: 1
Episode: 1120000, Mean Return: 4.692, Mean Wealth (V_T): 110.599, Epsilon: 1
Episode: 1130000, Mean Return: 4.691, Mean Wealth (V_T): 110.489, Epsilon: 1
Episode: 1140000, Mean Return: 4.689, Mean Wealth (V_T): 110.281, Epsilon: 1
Episode: 1150000, Mean Return: 4.691, Mean Wealth (V_T): 110.502, Epsilon: 1
Episode: 1160000, Mean Return: 4.691, Mean Wealth (V_T): 110.497, Epsilon: 1
Episode: 1170000, Mean Return: 4.688, Mean Wealth (V_T): 110.179, Epsilon: 1
Episode: 1180000, Mean Return: 4.692, Mean Wealth (V_T): 110.555, Epsilon: 1
Episode: 1190000, Mean Return: 4.693, Mean Wealth (V_T): 110.774, Epsilon: 1
Episode: 1200000, Mean Return: 4.694, Mean Wealth (V_T): 110.848, Epsilon: 1
Episode: 1210000, Mean Return: 4.693, Mean Wealth (V_T): 110.671, Epsilon: 1
E

Episode: 2160000, Mean Return: 4.692, Mean Wealth (V_T): 110.605, Epsilon: 1
Episode: 2170000, Mean Return: 4.691, Mean Wealth (V_T): 110.488, Epsilon: 1
Episode: 2180000, Mean Return: 4.691, Mean Wealth (V_T): 110.499, Epsilon: 1
Episode: 2190000, Mean Return: 4.692, Mean Wealth (V_T): 110.593, Epsilon: 1
Episode: 2200000, Mean Return: 4.693, Mean Wealth (V_T): 110.782, Epsilon: 1
Episode: 2210000, Mean Return: 4.692, Mean Wealth (V_T): 110.603, Epsilon: 1
Episode: 2220000, Mean Return: 4.692, Mean Wealth (V_T): 110.583, Epsilon: 1
Episode: 2230000, Mean Return: 4.69, Mean Wealth (V_T): 110.429, Epsilon: 1
Episode: 2240000, Mean Return: 4.692, Mean Wealth (V_T): 110.692, Epsilon: 1
Episode: 2250000, Mean Return: 4.694, Mean Wealth (V_T): 110.779, Epsilon: 1
Episode: 2260000, Mean Return: 4.691, Mean Wealth (V_T): 110.524, Epsilon: 1
Episode: 2270000, Mean Return: 4.691, Mean Wealth (V_T): 110.495, Epsilon: 1
Episode: 2280000, Mean Return: 4.692, Mean Wealth (V_T): 110.642, Epsilon: 1


Episode: 3230000, Mean Return: 4.692, Mean Wealth (V_T): 110.589, Epsilon: 1
Episode: 3240000, Mean Return: 4.693, Mean Wealth (V_T): 110.698, Epsilon: 1
Episode: 3250000, Mean Return: 4.689, Mean Wealth (V_T): 110.269, Epsilon: 1
Episode: 3260000, Mean Return: 4.69, Mean Wealth (V_T): 110.416, Epsilon: 1
Episode: 3270000, Mean Return: 4.691, Mean Wealth (V_T): 110.534, Epsilon: 1
Episode: 3280000, Mean Return: 4.693, Mean Wealth (V_T): 110.692, Epsilon: 1
Episode: 3290000, Mean Return: 4.692, Mean Wealth (V_T): 110.631, Epsilon: 1
Episode: 3300000, Mean Return: 4.692, Mean Wealth (V_T): 110.589, Epsilon: 1
Episode: 3310000, Mean Return: 4.691, Mean Wealth (V_T): 110.586, Epsilon: 1
Episode: 3320000, Mean Return: 4.69, Mean Wealth (V_T): 110.384, Epsilon: 1
Episode: 3330000, Mean Return: 4.691, Mean Wealth (V_T): 110.463, Epsilon: 1
Episode: 3340000, Mean Return: 4.69, Mean Wealth (V_T): 110.376, Epsilon: 1
Episode: 3350000, Mean Return: 4.692, Mean Wealth (V_T): 110.559, Epsilon: 1
Ep

Episode: 4300000, Mean Return: 4.693, Mean Wealth (V_T): 110.672, Epsilon: 1
Episode: 4310000, Mean Return: 4.693, Mean Wealth (V_T): 110.672, Epsilon: 1
Episode: 4320000, Mean Return: 4.692, Mean Wealth (V_T): 110.551, Epsilon: 1
Episode: 4330000, Mean Return: 4.693, Mean Wealth (V_T): 110.671, Epsilon: 1
Episode: 4340000, Mean Return: 4.691, Mean Wealth (V_T): 110.518, Epsilon: 1
Episode: 4350000, Mean Return: 4.692, Mean Wealth (V_T): 110.606, Epsilon: 1
Episode: 4360000, Mean Return: 4.691, Mean Wealth (V_T): 110.518, Epsilon: 1
Episode: 4370000, Mean Return: 4.692, Mean Wealth (V_T): 110.63, Epsilon: 1
Episode: 4380000, Mean Return: 4.693, Mean Wealth (V_T): 110.711, Epsilon: 1
Episode: 4390000, Mean Return: 4.689, Mean Wealth (V_T): 110.262, Epsilon: 1
Episode: 4400000, Mean Return: 4.693, Mean Wealth (V_T): 110.704, Epsilon: 1
Episode: 4410000, Mean Return: 4.689, Mean Wealth (V_T): 110.264, Epsilon: 1
Episode: 4420000, Mean Return: 4.691, Mean Wealth (V_T): 110.503, Epsilon: 1


...

Episode: 87030000, Mean Return: 4.694, Mean Wealth (V_T): 110.866, Epsilon: 1
Episode: 87040000, Mean Return: 4.69, Mean Wealth (V_T): 110.375, Epsilon: 1
Episode: 87050000, Mean Return: 4.692, Mean Wealth (V_T): 110.561, Epsilon: 1
Episode: 87060000, Mean Return: 4.69, Mean Wealth (V_T): 110.328, Epsilon: 1
Episode: 87070000, Mean Return: 4.689, Mean Wealth (V_T): 110.255, Epsilon: 1
Episode: 87080000, Mean Return: 4.69, Mean Wealth (V_T): 110.428, Epsilon: 1
Episode: 87090000, Mean Return: 4.693, Mean Wealth (V_T): 110.703, Epsilon: 1
Episode: 87100000, Mean Return: 4.69, Mean Wealth (V_T): 110.366, Epsilon: 1
Episode: 87110000, Mean Return: 4.691, Mean Wealth (V_T): 110.534, Epsilon: 1
Episode: 87120000, Mean Return: 4.694, Mean Wealth (V_T): 110.827, Epsilon: 1
Episode: 87130000, Mean Return: 4.693, Mean Wealth (V_T): 110.752, Epsilon: 1
Episode: 87140000, Mean Return: 4.689, Mean Wealth (V_T): 110.302, Epsilon: 1
Episode: 87150000, Mean Return: 4.691, Mean Wealth (V_T): 110.547, E

Episode: 88090000, Mean Return: 4.695, Mean Wealth (V_T): 110.919, Epsilon: 1
Episode: 88100000, Mean Return: 4.689, Mean Wealth (V_T): 110.285, Epsilon: 1
Episode: 88110000, Mean Return: 4.693, Mean Wealth (V_T): 110.656, Epsilon: 1
Episode: 88120000, Mean Return: 4.691, Mean Wealth (V_T): 110.48, Epsilon: 1
Episode: 88130000, Mean Return: 4.691, Mean Wealth (V_T): 110.503, Epsilon: 1
Episode: 88140000, Mean Return: 4.69, Mean Wealth (V_T): 110.426, Epsilon: 1
Episode: 88150000, Mean Return: 4.691, Mean Wealth (V_T): 110.527, Epsilon: 1
Episode: 88160000, Mean Return: 4.689, Mean Wealth (V_T): 110.29, Epsilon: 1
Episode: 88170000, Mean Return: 4.69, Mean Wealth (V_T): 110.358, Epsilon: 1
Episode: 88180000, Mean Return: 4.69, Mean Wealth (V_T): 110.468, Epsilon: 1
Episode: 88190000, Mean Return: 4.69, Mean Wealth (V_T): 110.421, Epsilon: 1
Episode: 88200000, Mean Return: 4.693, Mean Wealth (V_T): 110.633, Epsilon: 1
Episode: 88210000, Mean Return: 4.694, Mean Wealth (V_T): 110.785, Eps

Episode: 89150000, Mean Return: 4.692, Mean Wealth (V_T): 110.642, Epsilon: 1
Episode: 89160000, Mean Return: 4.694, Mean Wealth (V_T): 110.868, Epsilon: 1
Episode: 89170000, Mean Return: 4.693, Mean Wealth (V_T): 110.666, Epsilon: 1
Episode: 89180000, Mean Return: 4.689, Mean Wealth (V_T): 110.23, Epsilon: 1
Episode: 89190000, Mean Return: 4.692, Mean Wealth (V_T): 110.581, Epsilon: 1
Episode: 89200000, Mean Return: 4.692, Mean Wealth (V_T): 110.567, Epsilon: 1
Episode: 89210000, Mean Return: 4.692, Mean Wealth (V_T): 110.619, Epsilon: 1
Episode: 89220000, Mean Return: 4.691, Mean Wealth (V_T): 110.49, Epsilon: 1
Episode: 89230000, Mean Return: 4.691, Mean Wealth (V_T): 110.558, Epsilon: 1
Episode: 89240000, Mean Return: 4.691, Mean Wealth (V_T): 110.522, Epsilon: 1
Episode: 89250000, Mean Return: 4.69, Mean Wealth (V_T): 110.352, Epsilon: 1
Episode: 89260000, Mean Return: 4.691, Mean Wealth (V_T): 110.517, Epsilon: 1
Episode: 89270000, Mean Return: 4.693, Mean Wealth (V_T): 110.732, 

Episode: 90210000, Mean Return: 4.69, Mean Wealth (V_T): 110.314, Epsilon: 1
Episode: 90220000, Mean Return: 4.69, Mean Wealth (V_T): 110.415, Epsilon: 1
Episode: 90230000, Mean Return: 4.693, Mean Wealth (V_T): 110.704, Epsilon: 1
Episode: 90240000, Mean Return: 4.692, Mean Wealth (V_T): 110.658, Epsilon: 1
Episode: 90250000, Mean Return: 4.691, Mean Wealth (V_T): 110.49, Epsilon: 1
Episode: 90260000, Mean Return: 4.693, Mean Wealth (V_T): 110.811, Epsilon: 1
Episode: 90270000, Mean Return: 4.689, Mean Wealth (V_T): 110.353, Epsilon: 1
Episode: 90280000, Mean Return: 4.69, Mean Wealth (V_T): 110.37, Epsilon: 1
Episode: 90290000, Mean Return: 4.688, Mean Wealth (V_T): 110.212, Epsilon: 1
Episode: 90300000, Mean Return: 4.694, Mean Wealth (V_T): 110.809, Epsilon: 1
Episode: 90310000, Mean Return: 4.689, Mean Wealth (V_T): 110.305, Epsilon: 1
Episode: 90320000, Mean Return: 4.692, Mean Wealth (V_T): 110.603, Epsilon: 1
Episode: 90330000, Mean Return: 4.69, Mean Wealth (V_T): 110.421, Eps

Episode: 91270000, Mean Return: 4.691, Mean Wealth (V_T): 110.539, Epsilon: 1
Episode: 91280000, Mean Return: 4.691, Mean Wealth (V_T): 110.545, Epsilon: 1
Episode: 91290000, Mean Return: 4.692, Mean Wealth (V_T): 110.608, Epsilon: 1
Episode: 91300000, Mean Return: 4.693, Mean Wealth (V_T): 110.66, Epsilon: 1
Episode: 91310000, Mean Return: 4.688, Mean Wealth (V_T): 110.26, Epsilon: 1
Episode: 91320000, Mean Return: 4.695, Mean Wealth (V_T): 110.919, Epsilon: 1
Episode: 91330000, Mean Return: 4.692, Mean Wealth (V_T): 110.591, Epsilon: 1
Episode: 91340000, Mean Return: 4.691, Mean Wealth (V_T): 110.488, Epsilon: 1
Episode: 91350000, Mean Return: 4.692, Mean Wealth (V_T): 110.648, Epsilon: 1
Episode: 91360000, Mean Return: 4.693, Mean Wealth (V_T): 110.74, Epsilon: 1
Episode: 91370000, Mean Return: 4.692, Mean Wealth (V_T): 110.588, Epsilon: 1
Episode: 91380000, Mean Return: 4.69, Mean Wealth (V_T): 110.426, Epsilon: 1
Episode: 91390000, Mean Return: 4.689, Mean Wealth (V_T): 110.275, E

In [ ]:
# Print running time
print("Process time: {} s".format(t_1 - t_0))

In [ ]:
# Transforms the Q-table to interval representation
Q_interval = transform_Q_numeric_to_Q_interval(Q, wealth_bins)


# Prints the learned Action-values for each state + the best action for each state
for key in Q_interval.keys():
    print("Key:", key)
    print("State-Action Values:", Q_interval[key], sep="\n")
    print("Best Action (Investment in risky asset):", decode_action(np.argmax(Q_interval[key]), actions))

In [ ]:
def plot_q_values(Q, actions):
    '''Creates a 3d Wireframe plot of the Q-value function for each state-action pair 
       and adds the predicted action (i.e. argmax_a Q(s,a)
    
    Args:
    :params: Q [dict] A dictionary containing the action-values for each state
    :params: actions [np.array] A np.array containing the possible actions
    '''
    
    
    def fun(x, y, Q, actions, t):
        '''Help function used in plot_q_values'''
        # Returns the Q-values for each state-action pair at time step .
        return np.array([Q[(t,wealth)][encode_action(action, actions)] for action, wealth in zip(x,y)])
    
    times = sorted(list(set([t for t,_ in Q.keys()])))
    for time in times:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x = actions
        y = np.array(sorted([wealth for t, wealth in Q.keys() if t == time]))
        X, Y = np.meshgrid(x, y)
        zs = np.array(fun(np.ravel(X), np.ravel(Y), Q, actions, time))
        Z = zs.reshape(X.shape)

        # Predicted Actions for each state
        states = [key for key in Q.keys() if key[0] == time]
        predicted_actions = [decode_action(np.argmax(Q[state]), actions) for state in states]
        wealths = [wealth for _, wealth in states]
        predicted_Q_values = [Q[state][np.argmax(Q[state])] for state in states]

        ax.plot_wireframe(X, Y, Z, color="black")

        ax.set_xlabel('investment in risky asset')
        ax.set_ylabel('wealth')
        ax.set_zlabel('Q-values')
        ax.scatter(predicted_actions, wealths, predicted_Q_values, zdir="z", c="red", alpha=1, label="Predicted Actions")
        plt.title("Learned Q-value surface (at t={})".format(time))
        ax.legend()

        plt.show()

In [ ]:
plot_q_values(Q, actions)

In [ ]:
def plot_learned_vs_optimal_policy(env, Q, actions):
    '''Plots the learned policy derived from the action-value function Q vs. the optimal policy for logarithmic utility.
    
    Args:
    :params env[gym.environment]: The environment used for collecting samples.
    :params Q[dict]: The Q-Table.
    :params actions[np.array]: np.array containing the possible investment choices.
    '''
    
    # Construct a seperate plot for each time 
    for time in range(1, int(env.num_timesteps)):
        # get the observed wealth levels for each time
        wealth_levels = sorted([wealth for t, wealth in Q.keys() if t == time])
        # Derives the investment choice from the action-value function Q for the given state
        predicted_actions = [decode_action(np.argmax(Q[(time, wealth)]), actions) for wealth in wealth_levels]

        
        # Plots the learned policy
        plt.plot(wealth_levels, predicted_actions, label="learned")
        # Plots the optimal policy for logarithmic utility
        plt.plot(wealth_levels, ((env.mu - env.r)/(env.sigma**2))*np.ones(len(wealth_levels)), "-.", label="optimal")
        plt.title("Learned policy vs. optimal policy (at time t={})".format(time*env.dt))
        plt.xlabel("wealth")
        plt.ylabel("risky asset allocation")
        plt.legend()
        
        plt.show()

In [ ]:
plot_learned_vs_optimal_policy(env, Q, actions)